# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
#可以用document.documentElement.scrollHeight，如果body不行的話
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if last_height == new_height:
        print('到達頁面底端')
        break
    else:
        last_height = new_height
        print('網頁更新中...')

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 「台版李鍾碩」儲榢逸演出國片《海霧》　出道前自行創業有故事

▲儲榢逸跨足國片海霧演出。（圖／儲榢逸提供）
記者周康玉／台北報導
有「台版李鍾碩」之稱的儲榢逸，去年參與國片史上第一部以海上災難片為題的《海霧》，但他其實還有另一個身份，也就是特效化妝工作室的老闆，年營收大概三百萬，這一切，都是他自己打拚而來的。
儲榢逸18歲就勇闖電影特效化妝，在讀高職的時候，每天一下課就往西門町材料行跑，每晚在店裡自學，不賺一毛錢，就這樣整整「玩」了兩年，誤入這個特效化妝的叢林。
儲榢逸回憶，踏入這行遭受家人很大的阻礙，尤其媽媽是個很務實的傳統南部人，認為就是要唸有用的科系、然後賺錢，但儲榢逸還是依舊堅持跟著自己的興趣走，每天往材料店做「免錢的」。
母親對他的不務正業很不諒解，甚至鬧起家庭革命，有一次開車載著他到半途中，他堅持要去材料行練習特效妝，母親說：「你去就不要回來。」對於媽媽的不理解，他也氣到下車走人。儲榢逸說，像這樣的正面衝突至少有三次以上，直到他正式開業，也逐漸被媒體報導，媽媽才慢慢諒解。
儲榢逸的特效化妝漸漸在演藝圈傳開，合作的對象也越來越大咖，三年前還為星爺周星馳的《美人魚》特效化妝，以及強檔鬼片《紅衣小女孩》的鬼妝操刀。
儲榢逸目前的特效化妝工作室，去年營收約300萬元，受到兩岸情況不明朗，大陸案量銳減影響，比去年衰退三成，但僅僅三人小團隊的人力而言，算是維持得還可以。儘管如此，儲榢逸也沒閒著，從兩年前開始跨足演員和歌手，在海外四處巡演，除了開創更多元的演藝事業，同時還可維持演藝圈人脈，繼續開拓他特效化妝本業的市場。
儲榢逸在他的大銀幕處女作《海霧》片中，與男星鄭人碩、李李仁、連凱、李康生、王陽明等合作，電影場地也使用曾經支援拍攝李安導演的《少年ＰＩ奇幻旅程》的中台灣影視基地的造浪池、深水池等媲美好萊塢等級設備。
儲榢逸在片中飾演一名初出茅廬的船員，因為太想證明自己的能力，負氣離家跑船，卻因為人性險惡，遇上許多沒經歷的經歷冒險，更好幾度在理想過程中迷失自己。
身兼此劇的演員和特效化妝師，儲榢逸參與演出時，比過去僅擔任化妝師時，更要投入角色。儲榢逸為了要更入戲，好幾次找在海港工作的朋友深聊，揣摩船員的心境；也因應導演要

▲憂疫情創全球，道瓊重挫603點、美股長黑。（圖／路透，下同）
中央社／紐約31日綜合外電報導
投資人擔心新冠狀病毒疫情恐創擊全球經濟成長，美股今天重挫，隨全球主要股市走弱。
道瓊工業指數終場下跌603.41點，或2.09%，收在28256.03點；標準普爾500指數下跌58.14點，或1.77%，收在3225.52點。
以科技股為主的那斯達克指數下跌147.996點，或1.59%，收在9150.938點。
----------------------------------------------------------------------

[12] 義大利防疫禁飛　華航宣佈取消2班羅馬－台北航班

▲武漢肺炎疫情延燒全球，華航今（31）日晚間發布聲明表示，將取消2月2日台北（桃園）-羅馬CI75與2月3日羅馬-台北（桃園）CI76航班。（圖／華航提供）
記者林淑慧／台北報導
武漢肺炎疫情延燒全球，義大利今（31）日出現首例確診病例，並宣布禁止台灣、中國、香港、澳門航班往返，華航晚間發布聲明表示，已接獲義大利飛航通告，將取消2月2日台北（桃園）-羅馬CI75與2月3日羅馬-台北（桃園）CI76航班，後續航班若有調整將再行公告。
義大利衛生部門今天通報兩名中國大陸赴義旅客確診武漢肺炎，目前正在羅馬接受隔離治療，隨後義大利總理孔特宣佈，義大利將進入「國家緊急狀態」並將持續六個月，政府決議撥出500萬歐元，以因應新型冠狀病毒的疫情，因而衝擊我國籍航空航線與航班安排。
華航表示，中華航空接獲義大利飛航通告，因該國健康管理局要求，2月2日台北（桃園）-羅馬CI75與2月3日羅馬-台北（桃園）CI76航班取消，華航將協助已訂妥機位的旅客安排替代航班抵達目的地。
另外，即將於2月18日開航台北－米蘭航線的長榮航空，原訂每週飛航四班，受到義大利宣佈禁飛影響，對於開航是否延期，長榮航空目前仍在透過管道進行了解中。
依據華航資料，華航台北－羅馬航線每週飛航3班，週二、四、日從桃園出發，週一、三、五從羅馬返台；根據民航局統計，華航羅馬航線2018年共飛航260架次，載客數達6萬3141人次，載客率達79.4%。
----------------------------------------------------------------------

[13] 快訊／大樂透連續

▲中信銀行My Way簽帳金融卡首波香味特別版限量發行5萬張，共有4種獨特香味。（圖／中信銀行提供，下同）
記者紀佳妘／台北報導
中信銀行今（31）日宣布，My Way數位存款帳戶即日起推出「My Way簽帳金融卡」（My Way卡），首波「香味特別版」限量發行5萬張，共有4種獨特香味，卡片還隱藏AR（擴增實境）互動驚喜，隨手一掃立即體驗。此外，為搶攻旅遊族群，My Way卡祭出海外消費2.5%現金回饋無上限，也是2020年現金回饋最高的簽帳金融卡。
中信銀行表示，My Way卡的卡面以貓頭鷹「小B」為主角，透過線條勾勒出城市意象，搭配年輕族群喜愛的馬卡龍色系，讓小B帶領客戶環遊世界並享海外消費高額回饋。
中信銀行也以「Way」的諧音「味」為靈感，首推香味簽帳金融卡，將My Way卡結合4款獨特香味，包括香蕉、椰子、玫瑰、巧克力，讓客戶自行選擇「My味」，打造輕鬆愉悅氛圍。舉例來說，黃色的「蕉遊卡」散發香蕉甜美香氣，期待出遊的心情即刻浮現。
此外，My Way卡這次加碼酷炫科技，只要打開「中信Home Bank」APP掃描卡面，即可解密專屬AR動畫並進行趣味互動，吸引喜愛創新科技的消費者搶先入手。
中信銀行指出，只要申辦My Way數位存款帳戶成功即可擁有My Way卡，首波香味特別版採限量發行5萬張，歡迎新、舊客戶立即申辦，成功開立My Way數位存款帳戶並登入網路銀行，還可獲得網路商城購物金3,000元，完成指定任務再享現金回饋。
未持有中信銀行外幣帳戶的民眾開立外幣數位存款帳戶可享指定幣別換匯優惠，6月30日前再加碼單月一次外幣ATM提領或存款免手續費優惠，搭配My Way卡海外消費2.5%現金回饋，讓喜愛出國的民眾省上加省。
▼中信銀行My Way簽帳金融卡祭出海外消費2.5%現金回饋無上限。（圖／中信銀行提供）
----------------------------------------------------------------------

[25] 中鋼去年獲利128億元亞洲排第二　董座：「看好鋼市春燕成群飛來！」

▲中鋼集團董事長翁朝棟表示，預期今年鋼價及景氣可望逐步回升， 鋼市春燕即將成群飛來。(圖／中鋼提供）
記者林淑慧／台北報導
中美貿易戰持續延燒，鋼鐵市場遭遇逆風，中鋼集團董事長翁朝棟表示，中鋼去年稅前盈餘達128億元，在獲利方面

▲ 新台幣兌美元收盤升值2.7分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（31）天收盤升值2.7分、收在30.25元兌1美元，成交金額18.39億美元。
新台幣匯率今日開盤先以30.28元兌1美元、小貶0.3分開出，隨後走勢反轉，盤中幾度升值逾1角，最高來到30.16元兌1美元，最低則是30.28元兌1美元。
根據央行發布之主要貨幣對美元匯率變動資料顯示，今日新台幣升值0.09%，泰銖升值0.19%、歐元升值0.15%；韓元貶值0.57%、澳幣貶值0.24%、日圓也貶值0.17%。
另外，英國央行宣布不降息、美英貿易緊張局勢減弱、英國確定脫歐等多方消息影響，英鎊升值了0.98%。
----------------------------------------------------------------------

[37] 17款新春刮刮樂中獎機率報你知　還有546個百萬以上大獎待刮出

▲根據台彩統計，「2000萬超級紅包」刮刮樂目前只被刮出1個頭獎，還有2個未被刮出。（示意圖／記者李毓康攝）
記者紀佳妘／台北報導
台灣彩券公司今（2020）年推出17款新春刮刮樂，其中春節限定的「2000萬超級紅包」，頭獎2,000萬元吸引許多民眾試手氣。根據台彩統計，目前「2000萬超級紅包」頭獎還有2個機會，合計17款刮刮樂100萬元（含）以上的獎項仍有546個尚未刮出。
台灣彩券公司這次推出17款新春刮刮樂，總獎項超過4,060萬個，100萬元（含）以上的獎項共有646個，足足比2018年增加逾200個，其中深受民眾喜愛的「2000萬超級紅包」，今年再度加碼頭獎2000萬元有3個，二獎「200萬元加賓士車」共有7個，三獎100萬元則增加至400個，總中獎率達69.33%，總獎金逾74億元，也是歷年刮刮樂單期遊戲總獎金最高的一款。
根據台彩統計，到昨天（1月30日）為止，17款新春刮刮樂100萬元（含）以上的獎項已被刮出100個，其中100萬元刮出92個、200萬元刮出5個、300萬元刮出2個、500萬元（含）以上則被刮出1個。
換句話說，100萬元以上刮刮樂大獎只刮出100個，目前還有546個大獎尚未刮出，因此想靠刮刮樂成為百萬富翁的民眾還有機會。
所謂的500萬元（含）以上的獎項即是「2000萬超級紅包」的頭獎2,000萬元，目前已被

▲印度裔的Arvind Krishna，將接任IBM執行長。（圖／取自IBM）
記者謝仁傑／綜合報導
消息宣布後，IBM股價盤後交易上漲了約5％。Wedbush分析師表示，市場顯然對此期待。
克里希納（Arvind Krishna）是2018年IBM以340億美元（當時約新台幣1.05兆元）收購Red hat背後的策劃者，他向羅梅蒂和董事會提出了收購建議，指出混合雲端是IBM未來成長的最佳選擇。克里希納也曾擔任IBM軟體戰略副總裁，他也領導了許多新技術的開發，包括人工智慧、雲端計算和量子計算，並開創了IBM的安全軟體業務。
他將成為繼微軟的納德拉（Satya Nadella）、Google及Alphabet的皮查伊（Sundar Pichai）、Adobe的納拉延（Shantanu Narayen）之後，第四位美國科技巨頭的印度裔執行長。
▲IBM現任執行長Ginni Rometty。（圖／路透社）
事實上，自從2012年羅梅蒂接任IBM執行長以來，公司股價和銷售額均下跌超過25％；當年底，IBM市值仍超過2000億美元，收入超過1000億美元。如今眾多科技巨頭正在快速發展，而IBM在她的領導下卻顯得相當艱難。
----------------------------------------------------------------------

[50] 未延期！會計師提醒　扣繳憑單申報至下周三止

▲ 國稅局示意圖（圖／記者李鍾泉攝）
記者吳靜君／台北報導
今年1月份因逢春節假期，辦理2019年度的各類所得扣繳申報，包含扣繳暨免扣繳憑單、信託財產各類所得憑單及股利憑單申報，期限已經調整為109年1月1日至2月5日，KPMG安侯建業聯合會計師事務所稅務投資部會計師游雅絜提醒，目前並不會因為碰到武漢肺炎疫情而可延期申報。
KPMG安侯建業表示，依據國稅局的說明，各類所得憑單填發單位若於規定申報期限內採用網路申報，且憑單上所載之納稅義務人為在中華民國境內居住的個人，填發單位只需於系統內憑單填發方式勾選「免填發」或「電子憑單」，可免填發各式憑單予納稅義務人，可節省紙張使用量及減少郵寄及印發成本，節能減紙以響應環保。
游雅絜強調，扣繳義務人雖已依稅法規定扣繳稅款，仍要記得依規定期限按實填報或填發扣繳憑單，不然稅捐稽徵機關除會依法要求限期補報或填發之外，會再按扣繳稅額

▲台電將於5月再招募728名僱用人員，其中配電線路維護類379名為最多的類別。（圖／台電提供）
記者林淑慧／台北報導
年後轉職潮蠢動，國營事業有新職缺對外招手。台電今（31）日發佈今年首波新進人員甄試簡章，將在五月招募728名僱用人員，包括配電線路維護、機械運轉維護及綜合行政等19類別，正式錄取員工僱用起薪為31,487元，歡迎有志者加入台電大家庭。
依據台電「109年度新進僱用人員甄試簡章」，台電將於5月31日在臺北、臺中、高雄及花蓮四地同時舉行初（筆）試，高中職畢業即可報考，報名期間自2月11日至24日止；官員提醒，今年輸配變電等5類別考生將加考握力測試。
▲台電將於5月再招募728名僱用人員，為強化工作現場電線裁剪、導線壓接等實務需要，輸配變電等5類別考生將加考握力測試。（圖／台電提供）
台電表示，為因應企業永續經營並充實現場技術人力，台電持續招考新進人員，繼去(2019)年8月招募826名新進僱用人員，今年再續招19類別共728名，其中配電線路維護類379名最多，其次為機械運轉維護類66名、變電設備維護類55名、綜合行政類46名等。
另外，台電今年首波的新進僱用人員甄試，也首度招考輸電土建工程類人員，未來錄取者將投入輸變電工程單位架空、電纜線路及變電所土建工程等相關工作，另一新增之車床類報考者則需具備銑床或車床技術士證。
台電說明，最終錄取人員須經1年內部訓練，正式僱用起薪31,487元。甄試簡章、相關資訊及工作性質說明影片可上台電甄試網站(recruit.taipower.com.tw)查詢，歡迎具高中職以上學歷考生報考。
----------------------------------------------------------------------

[62] 上市公司公布107年董事酬金　被動元件龍頭國巨以1.14億奪冠

▲ 台灣證券交易所。（圖／記者李瑞瑾攝）
記者李瑞瑾／台北報導
臺灣證券交易所今（31）日公布上市公司董監事實際支領酬薪，據平均每位董事酬金排行，被動元件龍頭國巨以1.14億元奪冠，第二名則是中信金的5429.8萬元，第三是台積電的4013.7萬元。
第四到十名則分別為上銀2903.2萬、大立光2563.9萬元、統一2464萬元、華新科2352.2萬元、台新金1905.5萬元、和泰車1877.5萬元、遠東新1818.5萬

▲財政部長蘇建榮。（圖／記者屠惠剛攝）
記者吳靜君／台北報導
台股昨日開紅盤，創下歷史最大跌點，財長蘇建榮表示今（31）日在盤前召開台股開盤前股市信心喊話記者會，以三點信心喊話，包含國內的經濟景氣良好、國際股市已經穩定、以及國安基金會密切關注。
武漢肺炎疫情擴散，衝擊台股昨（30）日開紅盤日重挫696點或5.75%，不僅創最慘開紅盤日，更創下台股有史以來單日最大跌點紀錄。
蘇建榮盤前特別召開記者會，希望可以維持台股的穩定，他有三點宣示：
一，國內基本面良好，除了主計總處今年經濟成長率2.72%，加上台商回台投資的動能還在，包含經濟部的台商累計申請投資已經達到8600多億元、以及依據境外資金回台投資管理條例申請匯回的金額已經達到525億元；加上前瞻基礎建設將進入高峰期，對於台灣的經濟增添動能。
國際研究機構IHS Markit也預測，台灣2019年經濟成長率為2.7％，2020年也有望維持2.2％（根據國發會官網，2020年已上修為2.4％)
二，國際股市也已經穩定，蘇建榮表示，國際股市確實在1月23日武漢封城之後，確實有動盪，但是歐美股市已經穩定下來，美國股市昨日除了費城半導體之外，其餘皆有上漲、日韓股市下跌情況也已經收斂；英國股市受到英國成功脫歐的因素略有下跌。
至於中國大陸股市預計在下周一(2月3日)開盤，財政部也會隨時關注動態。
▲財政部長蘇建榮今日盤前召開記者會向國人信心喊話。（圖／記者吳靜君攝）
三，台股昨日雖然下跌696點，創下歷史最大跌點，但是昨日的成交量是放大的，上市公司的成交值有2754.16 億元；相較於過去日平均成交值約1500、1600億元，成交金額放大顯示股市動能還在。
財政部會身為國安基金的幕僚單位，隨時關注國內外股市的變化，股市如果有失序的情況，會做出必要的判斷，報請行政院國安基金委員會召開委員會議、聽取委員的意見採取必要的措施，所以民眾不要對股市過度恐慌。
----------------------------------------------------------------------

[75] 美股道瓊、那指反彈　激勵台指期開盤反攻上揚88點

▲美股道瓊上漲 124.99 點。回頭看台指期今（31）日開盤上漲 341 點，來到 11,460 點。（圖／路透）
記者陳心怡／台北報導
受到武漢肺炎疫情影響，台股昨天開市第一

記者許力方／台北報導
事實上，今年元旦第二天就開出首個威力彩頭獎，獎金高達3.41億元，獎落新北市土城區永安街的「投彩商行」，一注獨得，如今連槓7期，台彩並於1月20日加碼2億元，累積頭獎金額達3.8億元，攀升至今年最高頭獎金額。
►「秒殺小魚乾起司條」～最狂追劇零食來了！
----------------------------------------------------------------------

[87] 台灣金融業在大陸曝險金額1.99兆　金管會緊盯武漢肺炎疫情

▲根據金管會統計，國內銀行、保險、證券業對大陸曝險部位達1.99兆元。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
金管會今（30）日公布台灣金融業在大陸的曝險情況，到2019年11月底為止，國銀對大陸曝險金額降至1.71兆元，其中又以銀行授信業務減少531億元最多，若加上證券期貨業、保險業，累計3大金融業對大陸曝險額則為1.99兆元。
根據金管會銀行局統計，2019年11月底，國銀在大陸曝險金額為1.71兆元，占決算後淨值48.0%；2019年第1季、第2季、第3季對陸曝險占淨值比重依序為51.2%、49.9%、48.7%，呈現「按季遞減」，預期12月有機會跌破48%，可望創下史上最低。
統計指出，截至2019年國銀對大陸授信金額為1兆2,649億元、投資為4,170億元、資金拆存為1,696億元，與2018年12月同期相比，授信減少531億元、投資減少51億元、資金拆存減少71億元。
金管會銀行局分析，國銀對大陸曝險逐季下降的因素，主要是授信減少為主，這幾年大陸經濟轉型、下行中，面臨許多國際情勢的問題，因此台資銀行在授信業務上更為保守，至於曝險占淨值比重則是曝險下降、淨值上升所致。
至於證券期貨業部分，到2019年12月底總曝險金額154.31億元。證期局表示，目前有405家上市櫃公司發布重訊，揭露受到武漢肺炎疫情影響及因應措施，部分是配合當地政府防疫，也有部分公司則是調整生產線，不過這些公司所發布的重大訊息，尚未有重大影響。
另據保險局統計，保險業對大陸曝險金額為2,648億元，占可運用資金0.99%，其中壽險業為2,604億元、產險為44億元。
-------------------------------------------------------------------